In [1]:
from datasets import load_dataset
from transformers import MT5Tokenizer, MT5ForConditionalGeneration, MT5Config
import torch
from transformers.utils import is_torch_fx_proxy
from MT6 import MT6
from transformers import MT5TokenizerFast, T5ForConditionalGeneration
from noise_functions.MT6NoiseFunction import MT6NoiseFunction

<frozen importlib._bootstrap>:219: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


In [6]:
from transformers import T5Config

tokenizer = MT5TokenizerFast.from_pretrained("nikodallanoce/mt5-cc4-vanilla-32k-5")

cuda_dev = "cpu"

model = MT5ForConditionalGeneration.from_pretrained("/home/n.dallanoce/PyCharm/pretraining/weights/mt6_pre_en-fr(M1)_delta/checkpoint-100000")
#model = T5ForConditionalGeneration(MT5Config(vocab_size=len(tokenizer), tie_word_embeddings=False, dense_act_fn = "gelu_new",  feed_forward_proj= "gated-gelu"))
# model = MT6(
#         MT5Config(num_layers=6, d_model=512, num_heads=8, d_ff=2048, vocab_size=len(tokenizer), max_length=128,
#                   tie_word_embeddings=True))
model = model.to(cuda_dev)
model.train(False)
model.num_parameters()

You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.


76935552

In [15]:
model.config

MT5Config {
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.27.2",
  "use_cache": true,
  "vocab_size": 32103
}

In [15]:
new_config = T5ForConditionalGeneration.from_pretrained("google/t5-v1_1-small").config
new_config.vocab_size = len(tokenizer)
new_config

T5Config {
  "_name_or_path": "google/t5-v1_1-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.27.2",
  "use_cache": true,
  "vocab_size": 32103
}

In [22]:
from transformers import T5TokenizerFast

tokenizer = T5TokenizerFast.from_pretrained("t5-small")

cuda_dev = "cpu"

model = T5ForConditionalGeneration.from_pretrained(
    "t5-small", max_length=128, is_gated_act=True, tie_word_embeddings=True, feed_forward_proj=True)
model = model.to(cuda_dev)

Some weights of the model checkpoint at t5-small were not used when initializing T5ForConditionalGeneration: ['decoder.block.1.layer.2.DenseReluDense.wi.weight', 'decoder.block.2.layer.2.DenseReluDense.wi.weight', 'encoder.block.0.layer.1.DenseReluDense.wi.weight', 'decoder.block.4.layer.2.DenseReluDense.wi.weight', 'encoder.block.4.layer.1.DenseReluDense.wi.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'encoder.block.1.layer.1.DenseReluDense.wi.weight', 'encoder.block.5.layer.1.DenseReluDense.wi.weight', 'encoder.block.2.layer.1.DenseReluDense.wi.weight', 'decoder.block.5.layer.2.DenseReluDense.wi.weight', 'decoder.block.3.layer.2.DenseReluDense.wi.weight', 'encoder.block.3.layer.1.DenseReluDense.wi.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

In [12]:
inp_tok = tokenizer("Translate English to German: ")
tokenizer.batch_decode(inp_tok["input_ids"])

['Translat', 'e', 'English', 'to', 'German', ':', '</s>']

In [5]:
model.shared.weight

Parameter containing:
tensor([[-1.2188e+00, -4.3750e+00, -7.9062e+00,  ...,  2.0469e+00,
          3.0938e+00, -6.5938e+00],
        [-1.4438e+01,  8.1250e+00, -1.1719e+00,  ...,  1.1562e+01,
          4.8438e+00,  9.1000e+01],
        [ 8.1250e+00,  3.6250e+00, -1.9453e+00,  ..., -4.6250e+00,
          1.3125e+01,  2.1375e+01],
        ...,
        [-4.5117e-01, -3.3594e-01, -3.8867e-01,  ..., -2.0996e-01,
         -2.0000e+00, -9.1406e-01],
        [-1.0234e+00, -8.0859e-01,  4.3555e-01,  ..., -5.9326e-02,
         -9.2188e-01, -9.2969e-01],
        [ 1.0078e+00,  1.5234e-01, -2.4902e-01,  ..., -1.8555e-01,
         -2.7148e-01,  1.7969e+00]], requires_grad=True)

In [6]:
model.lm_head.weight

Parameter containing:
tensor([[-0.6602,  0.1572,  0.1777,  ..., -0.4199, -0.5938, -0.9609],
        [-0.2480,  0.3672,  0.8125,  ..., -0.0371, -1.0156, -0.5508],
        [-0.3555, -0.3613, -0.0688,  ..., -0.3672,  0.4180, -0.1465],
        ...,
        [-0.6055,  0.1631,  0.1729,  ..., -0.3359, -0.5703, -0.9531],
        [-0.7070,  0.2041,  0.1914,  ..., -0.3418, -0.5625, -0.9219],
        [-0.6914,  0.1426,  0.1738,  ..., -0.4082, -0.5273, -0.9297]],
       requires_grad=True)

In [7]:
model = MT5ForConditionalGeneration(
    MT5Config(num_layers=6, d_model=512, num_heads=8, d_ff=2048, vocab_size=len(tokenizer), max_length=133,
              tie_word_embeddings=True))

In [8]:
#tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small", max_length = 128)
inp_tok = tokenizer("this <extra_id_0> is very hot.",
                    text_target="Das Haus ist wunderbar.", return_tensors="pt")
out_gen = model.generate(inp_tok['input_ids'], max_new_tokens=128)
tokenizer.batch_decode(out_gen, skip_special_tokens=True)

AttributeError: 'Tensor' object has no attribute 'tile'

In [23]:
loss = model(**inp_tok).loss
loss

tensor(10.2929, grad_fn=<NllLossBackward>)

In [21]:
model.config

T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 128,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_bea